In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.utils import resample
from sklearn.metrics import classification_report
import numpy as np
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

data = pd.read_excel('/content/drive/MyDrive/Zunaira/Daraz Labelled Review Dataset with Sentiments and Features.xlsx', index_col=[0])
stopwords = pd.read_csv('/content/drive/MyDrive/Zunaira/roman-stop-words.csv')
df = pd.read_excel('/content/drive/MyDrive/Zunaira/train.xlsx')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = df[df['Label'] == 'fake']
df['Label'] = df['Label'].map({'fake':'1'})
df

<ipython-input-12-2df6681af774>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label'] = df['Label'].map({'fake':'1'})


,Review,Label
0,thnk amazon fori tarseel ke liye. haan yeh beh...,1
1,kitni price ka mila ap ko daba pack mojeh daba...,1
2,AOA mere vivo y19 hai agr kisi ne infinix zero...,1
3,My Husband Bought This Box Packed Phone For Me...,1
4,Kisi bhai ny sale krna ha to contact me from t...,1
...,...,...
936,mein you 12 khareedna chahta hon aur is ke ila...,1
937,Sony Xperia z3 compact sim working PTA not app...,1
938,mein htc u12 plus beech raha hon. . siyah 6 / ...,1
939,Hi every One .. plz dont deal with this guy .....,1


In [ ]:
data = data[['Reviews','Spam(1) and Not Spam(0)']].rename(columns={"Reviews":"Review","Spam(1) and Not Spam(0)":"Label"}).append(df).reset_index(drop=True)

<ipython-input-13-5433b6fbe28c>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data[['Reviews','Spam(1) and Not Spam(0)']].rename(columns={"Reviews":"Review","Spam(1) and Not Spam(0)":"Label"}).append(df).reset_index(drop=True)


In [ ]:
X = data['Review']
y = data['Label']

minority_indices = np.where(y == 1)[0]
majority_indices = np.where(y == 0)[0]

oversampled_minority_indices = resample(
    minority_indices,
    replace=True,
    n_samples=len(majority_indices),
    random_state=42
)

oversampled_indices = np.concatenate([majority_indices, oversampled_minority_indices])

X_oversampled = X[oversampled_indices]
y_oversampled = y[oversampled_indices]

In [ ]:
data.head()

data = X_oversampled.values
roman = set(stopwords['roman'])

In [ ]:
def preprocess_dataset(dataset):
    cleaned_dataset = [re.sub(r"[^a-zA-Z0-9\s]", "", text) for text in dataset]
    tokenized_dataset = [word_tokenize(text) for text in cleaned_dataset]
    lowercased_dataset = [[word.lower() for word in text] for text in tokenized_dataset]

    stop_words = roman
    preprocessed_dataset = [[word for word in text if word not in stop_words] for text in lowercased_dataset]

    return preprocessed_dataset

preprocessed_dataset = preprocess_dataset(data)

print("Preprocessed dataset:")
for text in preprocessed_dataset:
    print(text)

Streaming output truncated to the last 5000 lines.
['litter', 'acha', 'tareeqa', 'se', 'kaam', 'kerta', 'hai', 'iss', 'mein', 'dost', 'issue', 'hai', 'ke', 'billion', 'mein', 'saans', 'bemaari', 'paida', 'ker']
['maine', 'dosray', 'litter', 'isetmaal', 'kiye', 'magar', 'best', 'hai', 'qeemat', 'dair', 'barhi', 'hai', 'acha', 'hai', 'iss', 'mein', 'zara', 'si', 'badboo', 'nhi', 'hai', 'iss', 'ke', 'daana', 'size', 'munasib', 'hai']
['ek', 'dum', 'pathar', 'ban', 'hai', 'bas', 'iss', 'mein', 'thora', 'sa', 'dost', 'issue', 'hai']
['mayar', 'qeemat', 'free', 'scoop', 'behtareen', 'hai']
['parago', 'toffees', 'zaiqa', 'mein', 'awla']
['behtareen', 'mayar', 'munasib', 'qeemat', 'mein', 'mujhe', 'pasand']
['maine', 'order', 'kiye', 'magar', 'mile', 'mayar', 'bilkul', 'bekar', 'hai']
['daliya', 'zaiqa', 'mein', 'acha', 'hai', 'iss', 'bilkul', 'khareedain']
['fesad', 'khalis', 'shehad', 'munasib', 'qeemat', 'mein']
['tasalibaksh', 'mayar', 'zaiqa', 'munasib', 'qeemat', 'mein']
['mayar', 'zaiqa

In [1]:
targets = y_oversampled.values

NameError: ignored

In [ ]:
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec

def train_and_save_glove_model(corpus, embedding_dim=200, window_size=5, output_file="glove_model.bin"):
    w2v_model = Word2Vec(corpus, vector_size=embedding_dim, window=window_size, min_count=1, workers=4)
    word_vectors = w2v_model.wv
    word_vectors.save(output_file)

train_and_save_glove_model(preprocessed_dataset)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, SimpleRNN, Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors

def train_bi_lstm_model(corpus, target, glove_model_file="glove_model.bin"):
    glove_model = KeyedVectors.load(glove_model_file)
    embeddings_index = glove_model.key_to_index
    embedding_dim = glove_model.vector_size
    embedding_matrix = np.zeros((len(embeddings_index), embedding_dim))
    word_index = {word: i for i, word in enumerate(embeddings_index)}
    for word, i in word_index.items():
        if i < len(embeddings_index):
            embedding_vector = embeddings_index[word]
            embedding_matrix[i] = embedding_vector

    max_sequence_length = max(len(sent) for sent in corpus)
    num_classes = len(set(target))

    X = np.zeros((len(corpus), max_sequence_length), dtype=np.int32)
    y = np.zeros((len(target), num_classes), dtype=np.int32)

    for i, sent in enumerate(corpus):
        for j, word in enumerate(sent):
            X[i, j] = word_index[word]
        y[i] = target[i]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    model = Sequential()
    model.add(Embedding(len(embedding_matrix), embedding_dim, weights=[embedding_matrix],
                        input_length=max_sequence_length, trainable=False))
    model.add(Bidirectional(SimpleRNN(128, kernel_regularizer=l2(0.01))))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32, verbose=1)

    return model

In [ ]:
model = train_bi_lstm_model(preprocessed_dataset, targets, glove_model_file="glove_model.bin")

Epoch 1/50
170/170 [==============================] - 24s 127ms/step - loss: 2.1440 - accuracy: 0.4954 - val_loss: 1.2308 - val_accuracy: 0.9573
Epoch 2/50
170/170 [==============================] - 21s 122ms/step - loss: 1.1654 - accuracy: 0.4947 - val_loss: 0.9508 - val_accuracy: 0.0022
Epoch 3/50
170/170 [==============================] - 20s 116ms/step - loss: 0.9557 - accuracy: 0.5064 - val_loss: 0.8658 - val_accuracy: 0.0898
Epoch 4/50
170/170 [==============================] - 22s 127ms/step - loss: 0.8746 - accuracy: 0.4994 - val_loss: 0.8385 - val_accuracy: 0.6635
Epoch 5/50
170/170 [==============================] - 20s 116ms/step - loss: 0.8296 - accuracy: 0.4972 - val_loss: 0.8046 - val_accuracy: 0.1001
Epoch 6/50
170/170 [==============================] - 22s 128ms/step - loss: 0.8071 - accuracy: 0.5002 - val_loss: 0.8039 - val_accuracy: 0.2651
Epoch 7/50
170/170 [==============================] - 20s 116ms/step - loss: 0.8042 - accuracy: 0.4976 - val_loss: 0.7916 - val_ac

# DCF IMPLEMENTATION

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
def prepare_train_test_data(texts, targets, test_size=0.2):
    label_encoder = LabelEncoder()
    encoded_targets = label_encoder.fit_transform(targets)

    X_train, X_test, y_train, y_test = train_test_split(texts, encoded_targets, test_size=test_size, random_state=42)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)

    X_train_sequences = tokenizer.texts_to_sequences(X_train)
    X_test_sequences = tokenizer.texts_to_sequences(X_test)

    max_sequence_length = max(len(seq) for seq in X_train_sequences)
    X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

    return X_train_padded, X_test_padded, np.array(y_train), np.array(y_test), tokenizer,max_sequence_length

In [ ]:
X_train, X_test, y_train, y_test, tokenizer, max_len = prepare_train_test_data(X_oversampled.values, targets)

In [ ]:
def train_dcf_model(X_train, y_train, X_test, y_test):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    intermediate_layer_model = Sequential()
    intermediate_layer_model.add(model.layers[0])
    intermediate_output = intermediate_layer_model.predict(X_train)
    intermediate_output = intermediate_output.reshape(intermediate_output.shape[0], -1)

    X_combined = np.concatenate((intermediate_output, y_train.reshape(-1, 1)), axis=1)

    rf1 = RandomForestClassifier(n_estimators=100)
    rf2 = RandomForestClassifier(n_estimators=100)
    et1 = ExtraTreesClassifier(n_estimators=100)
    et2 = ExtraTreesClassifier(n_estimators=100)

    rf1.fit(X_combined, y_train)
    rf2.fit(X_combined, y_train)
    et1.fit(X_combined, y_train)
    et2.fit(X_combined, y_train)

    intermediate_output_test = intermediate_layer_model.predict(X_test)
    intermediate_output_test = intermediate_output_test.reshape(intermediate_output_test.shape[0], -1)
    X_combined_test = np.concatenate((intermediate_output_test, y_test.reshape(-1, 1)), axis=1)

    rf1_preds = rf1.predict_proba(X_combined_test)[:, 1]
    rf2_preds = rf2.predict_proba(X_combined_test)[:, 1]
    et1_preds = et1.predict_proba(X_combined_test)[:, 1]
    et2_preds = et2.predict_proba(X_combined_test)[:, 1]

    avg_probs = (rf1_preds + rf2_preds + et1_preds + et2_preds) / 4

    y_pred = np.round(avg_probs)

    accuracy = accuracy_score(y_test, y_pred)
    print("DCF Model Accuracy:", accuracy)

    return model


In [ ]:
def metrics(X_train, y_train, X_test, y_test):
    model = train_dcf_model(X_train, y_train, X_test, y_test)

    prediction = []
    y_pred = model.predict(X_test)
    for pred in y_pred:
        if pred[0] > 0.5:
            prediction.append(int(pred[0]))
        else:
            prediction.append(int(pred[0]))

    report = classification_report(y_test, prediction, target_names=['no-spam', 'spam'])
    print("Classification Report:")
    print(report)

    return model

model = metrics(X_train, y_train, X_test, y_test)

Epoch 1/10
170/170 [==============================] - 6s 27ms/step - loss: 4.0143 - accuracy: 0.5746 - val_loss: 0.8493 - val_accuracy: 0.5994
Epoch 2/10
170/170 [==============================] - 6s 36ms/step - loss: 0.6826 - accuracy: 0.6553 - val_loss: 0.6932 - val_accuracy: 0.5950
Epoch 3/10
170/170 [==============================] - 4s 25ms/step - loss: 0.5796 - accuracy: 0.6796 - val_loss: 0.6569 - val_accuracy: 0.6053
Epoch 4/10
170/170 [==============================] - 4s 25ms/step - loss: 0.5383 - accuracy: 0.6975 - val_loss: 0.6349 - val_accuracy: 0.6370
Epoch 5/10
170/170 [==============================] - 6s 36ms/step - loss: 0.5030 - accuracy: 0.7141 - val_loss: 0.6221 - val_accuracy: 0.6473
Epoch 6/10
170/170 [==============================] - 4s 25ms/step - loss: 0.4877 - accuracy: 0.7237 - val_loss: 0.6007 - val_accuracy: 0.6642
Epoch 7/10
170/170 [==============================] - 4s 25ms/step - loss: 0.4815 - accuracy: 0.7292 - val_loss: 0.5724 - val_accuracy: 0.6730

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
